In [ ]:
%matplotlib inline

# **PyTorch Quickstart**

이번 강좌에서는 신경망의 일반적인 작업들을 위한 PyTorch API를 살펴본다.  이 강좌는 [PyTorch 공식 튜토리얼](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)의 [한글 번역본](https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html)을 다시 부분적으로 수정한 것이다.

## 데이터 다루기

파이토치(PyTorch)는 **데이터 작업을 위한 2가지 기본 요소**인
``torch.utils.data.DataLoader`` 와 ``torch.utils.data.Dataset``를 제공한다. 
``Dataset``은 데이터와 정답(label)을 저장하고, ``DataLoader``는 ``Dataset``을 순회 가능한 객체(iterable)로 만들어준다.


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch는 [`TorchText`](https://pytorch.org/text/stable/index.html), [`TorchVision`](https://pytorch.org/vision/stable/index.html) 및
[`TorchAudio`](https://pytorch.org/audio/stable/index.html)와 같이 응용분야에 특화된 라이브러리를 다양한 데이터셋과 함께 제공한다.
이 강좌에서는 `TorchVision`이 제공하는 데이터셋 중에에 하나를 사용한다.

``torchvision.datasets`` 모듈은 CIFAR, COCO 등과 같은 다양한 실제 비전(vision) 데이터에 대한 ``Dataset`` (전체 목록은 [여기](https://pytorch.org/vision/stable/datasets.html)를 참조)을 포함하고 있다. 
이 튜토리얼에서는 그 중 `FasionMNIST` 데이터셋을 사용한다.

모든 TorchVision ``Dataset``은 매개변수 ``transform``과 ``target_transform``을 받아들이는데 이를 통해 원시(raw) 데이터와 정답(label)을 적절하게 전처리(preprocessing)하기 위한 절차를 지정할 수 있다. 여기에서는 전처리 절차로 `torchvision.transforms`이 제공하는 `ToTensor`를 적용하였다. `ToTensor`는 이미지의 픽셀값을 `0~1` 사이의 실수로 정규화하고, `Tensor` 데이터 타입으로 변환하는 일을 한다.


In [ ]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



``DataLoader``는 데이터셋을 순회 가능한 객체(iterable)로 감싸고(wrap), 자동화된 배치(batch), 샘플링(sampling),
섞기(shuffle) 및 다중 프로세스로 데이터 불러오기(multiprocess data loading)를 지원한다. 여기서는 배치 크기(batch size)를 64로 정의한다.
즉, 데이터로더(dataloader) 객체의 각 요소는 64개의 특징(feature)과 정답(label)을 묶음(batch)으로 반환한다.



In [ ]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: {}".format(X.shape))
    print("Shape of y: {} {}".format(y.shape, y.dtype))
    print(y)
    print(X.requires_grad, y.requires_grad)
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5])
False False


**Note:** 하나의 이미지의 `shape`이 `[1, 28, 28]`인 점에 주의하라. 보통의 Python 이미지 라이브러리에서는 이미지가 `[height, width, num_channels]`의 형태로 표현되는 것과 달리 PyTorch에서는 `[num_channels, height, width]`의 형태로 표현된다는 것을 보여준다. 

------------------------------------------------------------------------------------------

## 모델 만들기

PyTorch에서 신경망 모델은 일반적으로 [`nn.Module`](<https://pytorch.org/docs/stable/generated/torch.nn.Module.html>)을
상속받는 클래스(class)를 정의하여 생성한다. ``__init__`` 함수에서 신경망의 계층(layer)들을 정의하고, ``forward`` 함수에서
신경망에 데이터를 어떻게 전달할지 지정한다. 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(accelerate)한다.



In [ ]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu" 
#gpu 사용가능하면 cuda 설정하고 안 되면 cpu설정
print("Using {} device".format(device))

# 모델을 정의합니다. #평평한 레이어와 ReLU활성화 포함된 3개의 선형 레이어로 구성
class NeuralNetwork(nn.Module):
    def __init__(self):
      #super는 부모 클래스의 메소드를 호출하는데 사용되는 내장 파이썬 함수
      #nn.module의 초기화 메소드가 올바르게 실행되도록 하는데 사용
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
      #모든 차원의 요소를 하나의 차원으로 만드는 작업 -> 각 이미지의 모든 픽셀이 하나의 연속된 벡터로 표현
        x = self.flatten(x)
        #평탄화된 입력 텐서 x를 linear_relu_stack 연산에 전달 2개의 선형 레이어와 relu 활성화 함수로 구성
        #선형 레이어는 입력 텐서의 차원을 변경하고 relu 화렁화 함수는 비선형성을 추가하여 모델이 복잡한 패턴 학습하도록 해줌
        logits = self.linear_relu_stack(x)
        #logits 텐서 반환
        return logits

#NeuralNetwork는 nn.Module을 상속받아 만든 사용자 정의 신경망 모델, init, foward 포함
model = NeuralNetwork()

model = model.to(device) #move device use 'to' method
#model 구조 출력, 레이어와 구성 요소 확인 가능
print(model)

from torchsummary import summary #모델 요약 인쇄 및 매개변수 수 표시
print(summary(model, (1, 28, 28))) 
#모델에 대한 힙렵 모양은 (1,28,28)로 지정 입력 이미지가 높이와 너비가 28픽셀인 회색조
#레이어 유형 및 인덱스, 각 레이어의 출력 모양, 각 레이어의 매개 변수 수

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (M

In [ ]:
print(model.parameters())
#매개변수 메서드 사용하여 신경망 모델의 모든 매개변수 출력 
#-> 개별 매개변수를 얻기 위해 반복할 수 있는 생성기 객체 반환 됨.
for name, param in model.named_parameters():
  #모델의 모든 명명된 매개변수를 반복하여 각 매개변수의 데이터 텐서의 이름과 모양 출력
    # if param.requires_grad: 
    #-> 그라디언트가 필요한 매개변수의 이름과 모양만 출력, 훈련 중 업데이트 예정
    print(name, param.data.shape)


<generator object Module.parameters at 0x7fd2a6dbc3c0>
linear_relu_stack.0.weight torch.Size([512, 784])
linear_relu_stack.0.bias torch.Size([512])
linear_relu_stack.2.weight torch.Size([512, 512])
linear_relu_stack.2.bias torch.Size([512])
linear_relu_stack.4.weight torch.Size([10, 512])
linear_relu_stack.4.bias torch.Size([10])


첫 번째 선형 레이어의 가중치 및 바이어스 텐서 크기가 각각 (512,784) 및 (512,) ...
각기 최종 선형 레이어는 크기가 각각 (10,512) 및 (10,)인 가중치 및 바이어스 텐서가 있음

------------------------------------------------------------------------------------------




모델 매개변수 최적화하기
------------------------------------------------------------------------------------------
모델을 학습하려면 [손실 함수(loss function)](<https://pytorch.org/docs/stable/nn.html#loss-functions>)와
[옵티마이저(optimizer)](<https://pytorch.org/docs/stable/optim.html>)가 필요하다.



In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 학습 단계(training loop)에서 모델은 배치(batch)로 제공되는 학습 데이터셋에 대한 예측을 수행하고,
예측 오류를 역전파(backpropagate)하여 모델의 매개변수를 조정한다.



In [ ]:
#신경망 모델에 대한 훈련 루프 정의 입력 인수로 데이터로더, 모델, 손실 함수, 옵티마이저 받음
def train(dataloader, model, loss_fn, optimizer): 
    #데이터 로더에서 데이터 세트의 길이 검색
    size = len(dataloader.dataset) 
    #enumerate함수 사용하여 데이터 로더의 배치를 반복하여 배치 색인과 배치 데이터 가져옴
    for batch, (X, y) in enumerate(dataloader): 
      #배치 데이터는 to메소드 사용하여 장치로 이동
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        if batch == 0:
          print(pred)
          print(pred.shape)
          print(pred.dtype)

        # just for test
        if batch == 0:
          print(X.requires_grad)
          print(y.requires_grad)
          print(pred.requires_grad)
          print(loss.requires_grad)

        # 역전파
        #모델에 있는 모든 매개변수의 그래디언트를 재설정하기 위해 호출, 여러 역방향 패스에 누적되지 않음
        optimizer.zero_grad()
        #모델의 모든 매개변수에 대한 손실의 기울기 계산
        loss.backward()
        #계산된 기울기 사용하여 매개변수 업데이트
        optimizer.step()

        #현재 손실과 100 배치마다 처리된 샘플 수를 인쇄
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
#전반적으로 이 함수는 지정된 손실 함수 및 옵티마이저가 있는 데이터 세트에서 신경망 모델을 훈련하는데 사용
#딥러닝에서 일반적으로 사용되는 최적화 알고리즘인 미니 배치 확률적 경사 하강 방식으로 모델 매개변수 업데이트

모델이 학습하고 있는지를 확인하기 위해 테스트 데이터셋으로 모델의 성능을 확인해보는 함수를 작성한다.



In [ ]:
#신경망 모델에 대한 평가 루프 정의 데이터 로더, 모델 및 손실 함수를 입력 인수로 받음
def test(dataloader, model, loss_fn):
  #dataloader - 입력 데이터를 로드하고 전처리하는 책임, 훈련 및 평가 위해 제공
  #model - 학습 및 예측 담당 입력데이터 가져와서 일련의 수학적 연산을 수행하고 출력, 
  #뉴런계층으로 구성 가중치와 편향 조정하여 손실 함수 최소화하기 위해 최적화 알고리즘 사용하여 학습
  #요약하면 dataloader는 학습을 위해 model에게 데이터 제공, model은 데이터 처리 및 예측
    #데이터 로더에서 데이터 세트의 길이와 배치 수 검색
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    #모델을 평가모드로 설정, 이는 교육 및 평가 중에 다르게 동작하는 드롭아웃 및 배치 정규화와 같은 특정 계층을 해제하는데 필요
    model.eval()
    #총 테스트 손실 및 올바른 예측 수에 대한 변수 초기화
    test_loss, correct = 0, 0
    #평가 중 기울기 계산 되지 않도록 블록 사용하여 데이터로더 배치를 반복
    with torch.no_grad():
        for X, y in dataloader:
          #배치 데이터는 to 메서드 사용하여 장치로 이동
            X, y = X.to(device), y.to(device)
            #모델은 입력 배치에 대한 예측을 만드는데 사용
            pred = model(X)
            #예측과 실제 레이블 사이의 테스트 손실은 지정된 손실함수를 사용하여 계산
            test_loss += loss_fn(pred, y).item()
            #올바른 예측의 수는 예측에서 최대값의 인덱스를 실제 레이블과 비교한 다음 올바른 예측의 수를 합산하여 계싼
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    #총 테스트 손실과 올바른 예측을 각각 배치수와 데이터 세트 크기로 나누어 평균 테스트 손실과 정확도 계산
    test_loss /= num_batches
    correct /= size
    #테스트 오류, 정확도 및 평균 손실을 인쇄
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    #전반적으로 이 함수는 지정된 손실 함수가 있는 데이터 세트에서 훈련된 신경망 모델을 평가하는데 사용
    #모델 예측을 계산하고 데이터 세트에 대한 모델 정확도 평가
    #이는 새로운 보이지 않는 데이터에 대한 모델의 일반화 성능을 평가하는데 유용

학습은 여러번의 반복 단계(epochs)를 거쳐서 수행된다. 각 에폭마다 모델의 정확도(accuracy)와 손실(loss)을 출력한다; 에폭마다 정확도가 증가하고 손실이 감소하는 것을 보려고 한다.



In [ ]:
#지정된 에포크 수에 대해 신경망 모델을 훈련하고 각 에포크 후에 결과 인쇄
# 변수 epochs 5로 초기화
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    #train함수 호출하여 지정된 데이터 로더, 모델, 손실 함수 및 옵티마이저 사용하여 학습 데이터에서 모델을 학습시킴
    #train함수는 현재 데이터 배치에서 계산된 손실 및 기울기를 기반으로 모델 매개변수를 업데이트함
    train(train_dataloader, model, loss_fn, optimizer)
    #test함수는 테스트 데이터에서 모델의 평균 테스트 손실과 정확도 계산
    test(test_dataloader, model, loss_fn)
print("Done!")

#전반적으로 이 코드 블록은 고정된 수의 에포크 동안 신경망 모델을 훈련하고 각 에포크 후에 테스트 데이터에 대한 성능을 평가
#이를 통해 사용자는 시간이 지남에 따라 모델의 훈련 및 테스트 성능을 모니터링하고 필요에 따라 모델 또는 하이퍼파라미터를 조정할 수 있음

Epoch 1
-------------------------------
tensor([[ 1.7881e-03,  1.0141e-02, -2.9123e-02,  1.2636e-01,  1.8883e-02,
         -5.7652e-02, -1.1898e-02,  5.4571e-02, -5.8901e-02, -8.9472e-02],
        [-2.7487e-02, -1.0165e-02, -6.1572e-02,  1.3244e-01,  5.1912e-03,
         -6.4272e-02,  2.3242e-02,  5.7736e-02,  2.1097e-02, -1.8083e-01],
        [-1.7259e-02, -3.1118e-03, -6.7704e-02,  7.8826e-02,  2.5961e-02,
         -2.8806e-02, -6.9623e-03,  1.8339e-02,  1.9345e-03, -5.9357e-02],
        [-1.4828e-02, -6.0715e-03, -7.5352e-02,  9.0169e-02,  1.9161e-02,
         -2.0634e-02, -7.8087e-03,  1.9111e-02,  1.4523e-02, -7.5866e-02],
        [-1.6617e-02, -2.0482e-02, -7.2931e-02,  1.2790e-01,  2.9836e-02,
         -2.3602e-02,  1.3669e-02, -3.1353e-03,  9.6393e-03, -1.0617e-01],
        [-2.9845e-02,  3.3839e-02, -3.9985e-02,  1.1407e-01, -5.0885e-02,
         -6.1128e-02, -1.6323e-03,  7.5832e-02,  5.1114e-04, -1.2842e-01],
        [-2.9494e-02, -3.3312e-02, -7.7277e-02,  7.2270e-02,  2.23

------------------------------------------------------------------------------------------




모델 저장하기
------------------------------------------------------------------------------------------
모델을 저장하는 일반적인 방법은 (모델의 매개변수들을 포함하여) 내부 상태 사전(internal state dictionary)을
직렬화(serialize)하는 것이다.



In [ ]:
#훈련된 pytorch 신경망 모델의 상태 사전을 torch.save 함수 사용하여 파일에 저장
#model.state_dict 메서드는 모델 매개변수의 현재 상태를 포함하는 사전을 반환 여기에는 교육 중에 학습된 가중치와 편향이 포함
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


모델 불러오기
------------------------------------------------------------------------------------------

모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함된다.



In [ ]:
model = NeuralNetwork()
#model.load_state_dict 메서드는 상태 사전을 모델에 로드하는데 사용
#모델의 매개변수가 포함된 사전 개체를 사용하고 모델의 해당 매개변수를 업데이트한다
model.load_state_dict(torch.load("model.pth"))
#저장된 상태 사전에서 모델의 매개변수가 복원되면 모델 객체를 사용하여 모델을 처음부터 학습하지 않고도 새 입력 데이터에 대한 예측 수행 가능

<All keys matched successfully>

이제 이 모델을 사용해서 예측을 할 수 있다.



In [ ]:
#클래스 이름 목록 정의
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
#평가모드로 설정
model.eval()
#예시 이미지와 해당 레이블은 테스트 데이터세트에서 가져온다.
x, y = test_data[0][0], test_data[0][1]
print(x.shape)
with torch.no_grad():
  #model(x)사용하여 입력 이미지에 대한 클래스 확률 예측
    pred = model(x)
    #pred[0].argmax(0) 사용하여 확률이 가장 높은 클래스 선택
    #예측 및 설제 클래스 레입르은 각각 클래스 인덱스를 사용하여 클래스 이름 목록에서 조회
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

torch.Size([1, 28, 28])
Predicted: "Ankle boot", Actual: "Ankle boot"
